# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [34]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [35]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lesosibirsk,58.2358,92.4828,264.77,96,99,1.65,RU,1707255276
1,1,waitangi,-43.9535,-176.5597,292.49,89,97,2.24,NZ,1707254879
2,2,port-aux-francais,-49.3500,70.2167,279.20,90,100,11.14,TF,1707254882
3,3,hermanus,-34.4187,19.2345,292.48,79,36,7.01,ZA,1707255280
4,4,edinburgh of the seven seas,-37.0676,-12.3116,290.32,66,54,2.78,SH,1707255179


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [36]:
print(city_data_df.columns)


Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')


In [37]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(x='Lng', y='Lat', c='Humidity', 
                                      cmap='viridis', size='Humidity', 
                                      hover_cols=['City', 'Humidity'],
                                      title='City Humidity',
                                      xlabel='Longitude', ylabel='Latitude',
                                      geo=True, tiles='CartoLight')

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values

# Define ideal weather conditions
ideal_conditions = {
    'Max Temp': {'max': 264, 'min':215},  
    'Humidity': {'max':96 },             
    'Cloudiness': {'max':11},          
    'Wind Speed': {'max': 2}            
}

# Narrow down cities that fit the criteria
ideal_cities_df = city_data_df
for param, conditions in ideal_conditions.items():
    if 'min' in conditions:
        ideal_cities_df = ideal_cities_df.loc[ideal_cities_df[param] >= conditions['min']]
    if 'max' in conditions:
        ideal_cities_df = ideal_cities_df.loc[ideal_cities_df[param] <= conditions['max']]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
53,53,pingzhuang,42.0372,119.2889,262.40,72,1,0.95,CN,1707255334
371,371,berezovyy,51.6667,135.7000,250.21,95,5,1.36,RU,1707255687
509,509,dashiqiao,40.6373,122.5025,258.67,91,5,1.73,CN,1707255847


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df 

# Add an empty column, "Hotel Name," to df.
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
53,pingzhuang,CN,42.0372,119.2889,72,
371,berezovyy,RU,51.6667,135.7000,95,
509,dashiqiao,CN,40.6373,122.5025,91,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [31]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'radius': radius,
    'type': 'lodging',
    'limit': 10
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
 lat = row['Lat']
 lng = row['Lng']
    
# Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
params["filter"] = f"circle:{radius}@{lat},{lng}"
params["bias"] =  f"point:{lat},{lng}"

# Set base URL
base_url = "https://api.geoapify.com/v2/places"


# Make and API request using the params dictionaty
response = requests.get(base_url, params=params)
    
# Convert the API response to JSON format
name_address = response.json()
    
# Grab the first hotel from the results and store the name in the hotel_df DataFrame
try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
# Log the search results
print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
dashiqiao - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
53,pingzhuang,CN,42.0372,119.2889,72,
371,berezovyy,RU,51.6667,135.7000,95,
509,dashiqiao,CN,40.6373,122.5025,91,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display
# Configure the map plot with additional hover information
map_plot = hotel_df.hvplot.points(x='Lng', y='Lat', c='Humidity', 
                                   cmap='viridis', size='Humidity', 
                                   hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'],
                                   title='City Humidity with Nearest Hotel',
                                   xlabel='Longitude', ylabel='Latitude',
                                   geo=True, tiles='CartoLight')

# Display the map
map_plot



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)